In [45]:
# 根据lableme标注的图像和json文件，生成每一个object对应的mask图像的最小外接矩形框（白底，黑色矩形框）
import os
import json
import numpy as np
from PIL import Image, ImageDraw

# 定义函数获取每个物体的边界框
def get_masks_and_boxes(json_file, image_size):
    with open(json_file, 'r') as f:
        data = json.load(f)

    boxes = []
    img_width, img_height = image_size
    for shape in data['shapes']:
        label = shape['label']
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        points = shape['points']
        polygon = [(int(x), int(y)) for x, y in points]
        ImageDraw.Draw(Image.fromarray(mask)).polygon(polygon, outline=1, fill=1)
        boxes.append((min(polygon, key=lambda x: x[0])[0], min(polygon, key=lambda x: x[1])[1],
                      max(polygon, key=lambda x: x[0])[0], max(polygon, key=lambda x: x[1])[1]))
    return  boxes


# 定义函数保存边界框图像
def save_bbox_image(bbox, image_size, save_path):
    img = Image.new('L', image_size, 255)
    ImageDraw.Draw(img).rectangle(bbox, fill=0)
    img.save(save_path)

# 图像文件夹和JSON文件夹路径
image_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\image'
json_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\json'
output_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\bbox'

# 创建输出文件夹
os.makedirs(output_folder, exist_ok=True)

# 遍历图像文件夹中的图像文件
for image_file in os.listdir(image_folder):
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        image_path = os.path.join(image_folder, image_file)
        image_name = os.path.splitext(image_file)[0]
        json_file = os.path.join(json_folder, image_name + '.json')

        # 获取图像尺寸
        with Image.open(image_path) as img:
            image_size = img.size

        # 获取每个物体的边界框
        boxes = get_masks_and_boxes(json_file, image_size)

        # 保存每个物体的最小外接矩形框图像
        for i, bbox in enumerate(boxes):
            bbox_save_path = os.path.join(output_folder, f'{image_name}_bbox_{i}.png')
            save_bbox_image(bbox, image_size, bbox_save_path)


In [46]:
# 根据lableme标注的图像和json文件，生成每一个object对应的mask图像（白底，黑色填充）
import cv2
import json
import numpy as np
import os

def generate_masks(image_path, json_folder, output_folder):
    # 读取原始图像
    image = cv2.imread(image_path)

    # 创建输出文件夹（如果不存在）
    os.makedirs(output_folder, exist_ok=True)

    # 构建输出文件路径
    image_filename = os.path.basename(image_path)
    image_name = os.path.splitext(image_filename)[0]

    # 读取JSON文件
    json_file = os.path.join(json_folder, f'{image_name}.json')
    with open(json_file, 'r') as f:
        data = json.load(f)

    # 遍历每个物体
    for shape_idx, shape in enumerate(data['shapes']):
        # 提取物体的多边形坐标
        points = shape['points']
        points = np.array(points, dtype=np.int32)

        # 创建一个与原始图像大小相同的空白掩码图像
        mask = np.zeros_like(image)

        # 在掩码图像上填充物体的多边形区域
        cv2.fillPoly(mask, [points], (255, 255, 255))

        # 将掩码图像转换为灰度图像
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        # 将掩码图像反转，使背景为白色，物体区域为黑色
        mask_inverted = cv2.bitwise_not(mask_gray)

        # 生成保存路径
        mask_filename = f'{image_name}_{shape_idx + 1}_mask.jpg'
        mask_path = os.path.join(output_folder, mask_filename)

        # 保存掩码图像
        cv2.imwrite(mask_path, mask_inverted)

# 示例用法
output_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\mask'
image_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\image'
json_folder = r'C:\Users\15135\Desktop\ceshi\jiemian_fenge\json'

# 获取图像文件列表
image_files = os.listdir(image_folder)

# 遍历每张图像
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)

    # 生成掩码图像并保存
    generate_masks(image_path, json_folder, output_folder)
